### Imports

In [1]:
import os
import re
from typing import Dict

import mne
import numpy as np
import scipy.stats as sp_stats
from joblib import Parallel, delayed
from NirsLabProject.config.paths import Paths

from NirsLabProject.utils import pipeline_utils as putils
from NirsLabProject.config.consts import *

from NirsLabProject.utils import general_utils as utils
from NirsLabProject.config.consts import *
from NirsLabProject.config.subject import Subject
from NirsLabProject.utils.group_spikes import group_spikes
from NirsLabProject.utils import plotting


### Extract Paths of stimuli features
These are the features from the raw edf data without grouping, stimuli and sleeping features

In [2]:
subjects = [Subject(d, True) for d in os.listdir(Paths.products_data_dir_path) if d.startswith('p')]
stimuli_subjects = subjects
stimuli_subjects_with_features = filter(lambda subj: os.path.exists(subj.paths.subject_channels_spikes_features_path), stimuli_subjects)
stimuli_subjects_with_features = sorted(stimuli_subjects_with_features, key=lambda s: s.p_number)

In [3]:
def edit_coordinates(channels_spikes_features):
    channel_name_to_coordinates = utils.calculate_coordinates(subj)
    for channel, data in channels_spikes_features.items():
        x, y, z = utils.get_coordinates_of_channel(channel_name_to_coordinates, channel)
        data[:, CORD_X_INDEX] = x
        data[:, CORD_Y_INDEX] = y
        data[:, CORD_Z_INDEX] = z
    return channels_spikes_features

def regenerate_flat_features(subj: Subject):
    channels_spikes_features = np.load(
        subj.paths.subject_channels_spikes_features_path,
        allow_pickle=True
    ).item()
#     channels_spikes_features = edit_coordinates(channels_spikes_features)
    index_to_channel = putils.get_index_to_channel(subj, channels_spikes_features)
    groups, flat_features, _ = putils.group_spikes(subj, channels_spikes_features, index_to_channel)
    flat_features = utils.add_flag_of_scalp_detection_to_spikes_features(flat_features, None)
    flat_features = utils.add_stimuli_flag_to_spikes_features(subj, flat_features)
    flat_features = utils.add_sleeping_stage_flag_to_spike_features(subj, flat_features)
    subject_id = np.ones((flat_features.shape[0], 1)) * subj.p_number
    flat_features[:, SUBJECT_NUMBER] = subject_id.reshape(-1)
    return flat_features

for subj in stimuli_subjects_with_features:
    if subj.name in ['p485', 'p486', 'p488', 'p496', 'p499', 'p520']:
        flat_features = regenerate_flat_features(subj)
        print(subj.name, flat_features.shape)
        np.save(subj.paths.subject_flat_features_path, flat_features)
        subj.save_flat_features_to_csv()

Grouping spikes
p485 (19202, 22)
Grouping spikes
p486 (24306, 22)
Grouping spikes
p488 (86479, 22)
Grouping spikes
p496 (16361, 22)
Grouping spikes
p499 (58390, 22)
Grouping spikes
p520 (1011, 22)


In [9]:
import shutil
import os

for subj in stimuli_subjects_with_features:
    new_path = '/tmp/flat_features'
    if not os.path.exists(new_path):
        os.makedirs(new_path)
    shutil.copyfile(subj.paths.subject_flat_features_path, os.path.join(new_path, f'{subj.p_number}_flat_features.npy'))

In [5]:
import pandas as pd


def flat_feaures_to_dataframe(subject, subject_flat_features):
    titles = [ name for index, name in FEATURES_NAMES.items()]
    index_to_channel_name = np.load(subj.paths.subject_channel_name_to_index_path, allow_pickle=True).item()
    df = pd.DataFrame(subject_flat_features, columns=titles)
    index_to_channel_convert = lambda x: index_to_channel_name[float(str(x)[3:])]
    df['channel'] =  df['channel'].apply(index_to_channel_convert)
    df['group_focal'] =  df['group_focal'].apply(index_to_channel_convert)
    df['group'] =  df['group'].apply(lambda x: float(str(x)[3:]))
    df['is_in_scalp'] = df['is_in_scalp'].apply(lambda x: x == 1)
    return df

flat_feaures_to_dataframe(subj, flat_features)


,timestamp,channel,amplitude,duration,cord_x,cord_y,cord_z,group,group_focal,group_event_duration,...,group_event_shallowest,group_event_spatial_spread,is_in_scalp,stimuli_flag,hypnogram_flag,subject_number,group_focal_amplitude,angle,relative_spike_amplitude,relative_spike_duration
0,1761.0,LA2,2.976170,38.0,30.0000,3.333,-35.333,0.0,LA2,98.0,...,1.0,64.0,True,0.0,0.0,520.0,2.976170,3.477607,2.097570,25.0
1,1859.0,LA1,2.127265,20.0,-33.7500,0.750,-30.250,0.0,LA2,98.0,...,1.0,64.0,True,0.0,0.0,520.0,2.976170,4.503121,3.596624,28.0
2,2435.0,LA1,3.804194,24.0,-33.7500,0.750,-30.250,1.0,LA1,0.0,...,1.0,0.0,True,0.0,0.0,520.0,3.804194,13.968792,6.460579,34.0
3,3316.0,LA2,4.955938,29.0,30.0000,3.333,-35.333,2.0,LA2,0.0,...,1.0,0.0,True,0.0,0.0,520.0,4.955938,13.258108,4.473004,27.0
4,4408.0,LA4,8.316221,32.0,33.3333,-7.333,-22.000,3.0,LA4,0.0,...,1.0,0.0,True,0.0,0.0,520.0,8.316221,13.515828,8.457921,32.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1006,4676831.0,LA4,1.300367,41.0,33.3333,-7.333,-22.000,827.0,LA4,0.0,...,1.0,0.0,True,3.0,0.0,520.0,1.300367,1.823837,1.026618,36.0
1007,4677205.0,LA4,1.171589,37.0,33.3333,-7.333,-22.000,828.0,LA4,0.0,...,1.0,0.0,True,3.0,0.0,520.0,1.171589,2.174228,0.923803,31.0
1008,4682278.0,LA1,1.286573,48.0,-33.7500,0.750,-30.250,829.0,LA1,0.0,...,1.0,0.0,True,3.0,0.0,520.0,1.286573,1.430909,1.190395,46.0
1009,4737821.0,LA1,1.994857,51.0,-33.7500,0.750,-30.250,830.0,LA1,0.0,...,1.0,0.0,True,3.0,0.0,520.0,1.994857,1.798297,2.020678,52.0
